In [1]:
# Install dependencies
! pip install ta
! pip install tensorflow
! pip install keras

  Using cached tensorflow-2.3.0-cp38-cp38-win_amd64.whl (342.5 MB)
  Using cached tensorflow_estimator-2.3.0-py2.py3-none-any.whl (459 kB)
  Using cached gast-0.3.3-py2.py3-none-any.whl (9.7 kB)
  Using cached scipy-1.4.1-cp38-cp38-win_amd64.whl (31.0 MB)
  Using cached grpcio-1.32.0-cp38-cp38-win_amd64.whl (2.6 MB)
  Using cached tensorboard-2.3.0-py3-none-any.whl (6.8 MB)
  Using cached absl_py-0.10.0-py3-none-any.whl (127 kB)
  Using cached Keras_Preprocessing-1.1.2-py2.py3-none-any.whl (42 kB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached tensorboard_plugin_wit-1.7.0-py3-none-any.whl (779 kB)
  Using cached Markdown-3.2.2-py3-none-any.whl (88 kB)
  Using cached google_auth_oauthlib-0.4.1-py2.py3-none-any.whl (18 kB)
  Using cached google_auth-1.21.2-py2.py3-none-any.whl (93 kB)
  Using cached requests_oauthlib-1.3.0-py2.py3-none-any.whl (23 kB)
  Using cached cachetools-4.1.1-py3-none-any.whl (10

In [19]:
# Libraries for Data Analysis
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from datetime import timedelta

# Machine learning Library
from sklearn.preprocessing import RobustScaler

# Library for Performing Technical Analysis
import ta

# Tensorflow 
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout

# Loading the data
data_frame = pd.read_csv("SPY_historical_data.csv")

In [20]:
data_frame['Date'] = pd.to_datetime(data_frame.Date)
data_frame.set_index('Date', inplace=True)
data_frame.dropna(inplace=True)

data_frame = ta.add_all_ta_features(data_frame, open="Open", high="High", low="Low", close="Close", volume="Volume", fillna=True)

data_frame

C:\Users\ahadk\anaconda3\lib\site-packages\ta\trend.py:608: RuntimeWarning: invalid value encountered in double_scalars
  dip[i] = 100 * (self._dip[i]/self._trs[i])
C:\Users\ahadk\anaconda3\lib\site-packages\ta\trend.py:612: RuntimeWarning: invalid value encountered in double_scalars
  din[i] = 100 * (self._din[i]/self._trs[i])


,Open,High,Low,Close,Adj Close,Volume,volume_adi,volume_obv,volume_cmf,volume_fi,...,momentum_uo,momentum_stoch,momentum_stoch_signal,momentum_wr,momentum_ao,momentum_kama,momentum_roc,others_dr,others_dlr,others_cr
Date,,,,,,,,,,,,,,,,,,,,,
2015-09-17,200.020004,202.889999,199.279999,199.729996,180.822556,276046600,-2.072266e+08,276046600,-0.750694,0.000000e+00,...,0.000000,12.465291,12.465291,-87.534709,0.000000,199.729996,0.000000,-23.036361,0.000000,0.000000
2015-09-18,195.710007,198.679993,194.960007,195.449997,177.867691,223657500,-3.719645e+08,52389100,-0.744370,-9.572539e+08,...,5.847144,6.178947,9.322119,-93.821053,0.000000,197.667046,0.000000,-2.142892,-2.166186,-2.142892
2015-09-21,196.440002,197.679993,195.210007,196.460007,178.786850,105726200,-3.706798e+08,158115300,-0.612258,-8.052484e+08,...,16.036811,18.915530,12.519923,-81.084470,0.000000,197.118910,0.000000,0.516761,0.515431,-1.637205
2015-09-22,193.880005,194.460007,192.559998,193.910004,176.466248,153890900,-3.058837e+08,4224400,-0.402838,-7.462732e+08,...,20.949148,13.068789,12.721088,-86.931211,0.000000,195.823380,0.000000,-1.297976,-1.306473,-2.913930
2015-09-23,194.110001,194.669998,192.910004,193.600006,176.184082,92790600,-3.259176e+08,-88566200,-0.382482,-6.437720e+08,...,22.895233,10.067840,14.017386,-89.932160,0.000000,194.847903,0.000000,-0.159867,-0.159995,-3.069138
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-09-10,341.820007,342.529999,332.850006,333.890015,333.890015,90569500,1.036163e+10,3308245700,0.047201,-2.413177e+08,...,45.085510,4.015480,10.667177,-95.984520,4.381059,349.265031,-2.633267,-1.736365,-1.751616,67.170691
2020-09-11,335.820007,336.970001,331.000000,334.059998,334.059998,84680200,1.036376e+10,3392925900,0.052496,-2.047874e+08,...,43.675636,11.027020,13.917665,-88.972980,0.887971,348.391423,-2.923398,0.050910,0.050897,67.255798
2020-09-14,337.489990,340.380005,334.220001,338.459991,338.459991,65605700,1.038847e+10,3458531600,0.057425,-1.342943e+08,...,42.910760,26.882850,13.975117,-73.117150,-0.369380,347.965639,-2.621059,1.317127,1.308528,69.458768
